In [ ]:
#Making a Face Recognition System

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extraction(img):
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    
    if faces is ():
        return None
    
    for(x,y,w,h) in faces:
        cropped_face =img(y:y+h, x:x+w)
        
    return cropped_face

cap= cv2.VideoCapture(0)
count=0

while True:
    
    ret, frame =cap.read()
    if face_extractor(frame) is not None:
        count +=1
        face =cv2.resize(face_extractor(frame),(255,255))
        face=cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        
        file_name_path =('C://Users//ABHISHEK SHRIMAL//Desktop//linuxWorld//16May//Untitled Folder//data//validation' + str(count) + '.jpg')
        cv2.imwrite(file_name_path, face)
        
        cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Face Cropped', face)
        
    else:
        print("face Not Found")
        pass
    
    if cv2.waitKey(1) ==13 or count ==10:
        break
        

cap.release()
cv2.destroyAllWindows()
print("Image Samples Collected Successfully")

In [ ]:
from keras.applications import MobileNet

# MobileNet was designed to work on 224 x 224 pixel input images sizes
img_rows, img_cols = 224, 224 

# Re-loads the MobileNet model without the top or FC layers
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers 
# Layers are set to trainable as True by default
for layer in MobileNet.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

In [ ]:
def lw(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Set our class number to 3 (Young, Middle, Old)
num_classes = 2

FC_Head = lw(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = 'facedata/training/'
validation_data_dir = 'facedata/test/'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=45,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# set our batch size (typically on most mid tier systems we'll use 16-32)
batch_size = 32
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

                     
checkpoint = ModelCheckpoint("facerecognition.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

# Enter the number of training and validation samples here
nb_train_samples = 1097
nb_validation_samples = 272

# We only train 5 EPOCHS 
epochs = 5
batch_size = 16

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

In [ ]:
model.save('facerecognition.h5')

In [ ]:
from keras.models import load_model

classifier = load_model('facerecognition.h5')

In [ ]:
classifier=load_model("facerecognition.h5")
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

monkey_breeds_dict = {"[0]": "ABHISHEK SHRIMAL ", 
                      "[1]": "ROHIT SHARMA"
                     }

monkey_breeds_dict_n = {"ABHISHEK":"ABHISHEK SHRIMAL",
                        "ROHIT":"ROHIT SHARMA"
                       }

def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, monkey, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    

for i in range(0,10):
    input_im = getRandomImage("facedata/test/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3) 
    
    # Get Prediction
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
    draw_test("Prediction", res, input_original) 
    cv2.waitKey(0)

cv2.destroyAllWindows()

In [ ]:
from keras.models import load_model

model=load_model("facerecognition.h5")

import numpy as np
import os
import IPython.display as ipd
import webbrowser as wb
import cv2
from keras.preprocessing import image
import PIL

face_classifier =cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

def face_detector(img,size=0.5):
    faces=face_classifier.detectMultiScale(img,1.3,5)
    if faces is ():
        return img, []
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255,2))
        roi=img[y:y+h,x:x+w]
    return img,roi
        


In [ ]:
cap=cv2.VideoCapture(0)

while True:
    ret, frame =cap.read()
    image, face = face_detector(frame)
    try:
        face=np.array(face)
        face=np.expand_dims(face, axis=0)
        if(face.shape==(1,0)):
            cv2.putText(image,"Not Detected",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
            cv2.imshow("Face Recognition",image)
            print("3")
            pass
        
        else:
            results=model.predict(face)
            if(results[0][0] == 0.0):
                cv2.putText(image,"ROHIT SHARMA",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
                cv2.imshow("Face Recognition",image)
                print("1")
                
            elif(results[0][0] != 0.0):
                cv2.putText(image,"ABHISHEK SHRIMAL",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
                cv2.imshow("face Recognition",image)
                print("2")
    
    except:
        cv2.imshow("Face Recognition", image)
        cv2.putText(image,"No Face Found",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        cv2.putText(image,"Locked",(250,450),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2)
        print("4")
        pass
    if cv2.waitKey(1)==13:
        break
        
cap.release()
cv2.destroyAllWindows()
            